In [1]:
import pickle, json, time, datetime, pymongo, requests, copy, os, csv
from pymongo import MongoClient
import pandas as pd
from Bio import Medline
from Bio import Entrez

In [2]:
now=datetime.datetime.now()

In [3]:
nowtime=now.strftime('%Y/%m/%d')

In [4]:
print(nowtime)

2023/11/15


In [5]:
now_time=now.strftime('%Y_%m_%d')

In [6]:
nowtime="2023/09/30"
now_time="2023_09_30"

In [7]:
def cal_time(time):
    time = time.split('/')
    year = int(time[0])
    month = int(time[1]) - 12
    day = int(time[2])
    if month < 1:
        month += 12
        year -= 1
    time = datetime.datetime(year=year, month=month, day=1).strftime('%Y/%m/%d')
    return time

In [8]:
def cal_time2(time):
    time = time.split('/')
    year = int(time[0])
    month = int(time[1]) +1
    day = int(time[2])
    if month >12:
        month = 1
        year += 1
    time = datetime.datetime(year=year, month=month, day=1).strftime('%Y/%m/%d')
    return time

In [9]:
former_time=cal_time(nowtime)

In [10]:
former_time

'2022/09/01'

In [18]:
former_time='1951/01/01'

In [19]:
idlist=[]
while former_time < now_time:
    terms='("aging"[Title/Abstract] OR "ageing"[Title/Abstract] OR "longevity"[Title/Abstract] OR "centenarian"[Title/Abstract] OR "centenarians"[Title/Abstract] \
            OR "the elderly"[Title/Abstract] OR "the aged"[Title/Abstract] OR "old people"[Title/Abstract] OR "older people"[Title/Abstract] OR "old age"[Title/Abstract] \
            OR "gerontology"[Title/Abstract] OR "geroscience"[Title/Abstract] OR "lifespan"[Title/Abstract] OR "healthspan"[Title/Abstract] OR "life expectancy"[Title/Abstract] AND \
            Journal Article[ptyp] AND "humans"[MeSH Terms] AND English[lang] AND ("'+str(former_time)+'"[Date - Publication] : "'+str(cal_time2(former_time))+'"[Date - Publication]))'
    Entrez.email="12007002@zju.edu.cn"
    handle_0 = Entrez.esearch(db="pubmed", term=terms,retmax=10000)
    record = Entrez.read(handle_0)
    idlist+=list(record["IdList"])
    # print("Found %i results" % len(idlist))
    if len(list(record["IdList"]))==9999:
        print('error')
        break
    former_time=cal_time2(former_time)

In [20]:
len(idlist)

737717

In [30]:
len(set(idlist))

339918

In [32]:
idlist_list=list(set(idlist))

In [33]:
with open('idlist.txt','a') as f:
    for i in idlist_list:
        f.write(i)

In [30]:
client = MongoClient(username="hald",password="mclab236")

In [31]:
db = client.hald

In [32]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'hald')

# pubmed_raw_pmid

In [33]:
collections_pubmed_raw_pmid=db.pubmed_raw_pmid

In [34]:
pmids=list(collections_pubmed_raw_pmid.find())

In [21]:
len(pmids)

295579

In [37]:
pmids[:2]

[{'_id': ObjectId('6393476d5303aaecac1f079a'),
  'PMID': '35796512',
  'addtime': '20220724'},
 {'_id': ObjectId('6393476d5303aaecac1f079b'),
  'PMID': '35796396',
  'addtime': '20220724'}]

In [38]:
pmid_list=[]
for i in pmids:
    pmid_list.append(i.get('PMID'))

In [28]:
pmid_list[:2]

['35796512', '35796396']

In [29]:
len(pmid_list)

301892

In [39]:
len(pmid_list)

295579

In [34]:
pmid_dict={}
for i in pmids:
    pmid_dict[i.get('PMID')]=i.get('PMID')

In [35]:
len(pmid_dict)

301892

In [36]:
new_articles_raw=[]
for i in idlist_list:
    if not pmid_dict.get(i):
        new_articles_raw.append(i)

In [37]:
len(new_articles_raw)

38726

In [42]:
res=[]
for i in range(0,len(new_articles_raw),1000):
    id=new_articles_raw[i:i+1000]
    handle_1 = Entrez.efetch(db="pubmed", id=id, rettype="medline",retmode="text")
    records = Medline.parse(handle_1)
    res.append(list(records))

In [43]:
raw_pick=[]
for i in res:
    raw_pick+=i

In [44]:
len(raw_pick)

38726

In [45]:
reserved_raw_pick=[]
reserved_raw_pick_pmid=[]
for i in raw_pick:
    if i.get('AB'):
        if not pmid_dict.get(i.get('PMID')):
            i['addtime']=now_time
            reserved_raw_pick.append(i)
            reserved_raw_pick_pmid.append({'PMID':i.get('PMID'),'addtime':i.get('addtime')})

In [46]:
len(reserved_raw_pick)

37253

In [47]:
reserved_raw_pick[0]

{'PMID': '23778148',
 'OWN': 'NLM',
 'STAT': 'MEDLINE',
 'DCOM': '20140617',
 'LR': '20130827',
 'IS': '1873-2054 (Electronic) 1353-8292 (Linking)',
 'VI': '23',
 'DP': '2013 Sep',
 'TI': 'Small-area health comparisons using health-adjusted life expectancies: a Bayesian random-effects approach.',
 'PG': '70-8',
 'LID': 'S1353-8292(13)00063-4 [pii] 10.1016/j.healthplace.2013.04.003 [doi]',
 'AB': "Health-adjusted life expectancy (HALE) is one of the most attractive summary measures of population health. It provides balanced attention to fatal as well as non-fatal health outcomes, is sensitive to the severity of morbidity within the population, and can be readily compared between areas with very different population age structures. HALE, however, cannot be calculated at the small-area level using traditional life table methodology. Hence we propose a Bayesian random-effects modeling approach that recognizes correlations and pools strength between sexes, age-groups, geographical areas, an

In [48]:
collections_pubmed_raw_pmid.insert_many(reserved_raw_pick_pmid)

In [49]:
len(list(collections_pubmed_raw_pmid.find()))

339145

In [29]:
len(list(collections_pubmed_raw_pmid.find({'addtime':'20230930'})))

0

In [26]:
pmids[-1]

{'_id': ObjectId('6459f20c48e9c8c060125f41'),
 'PMID': '30772335',
 'addtime': '2023_05_09'}

In [27]:
pmids[0]

{'_id': ObjectId('6393476d5303aaecac1f079a'),
 'PMID': '35796512',
 'addtime': '20220724'}

In [28]:
pmids[-100]

{'_id': ObjectId('6459f20c48e9c8c060125ede'),
 'PMID': '35341422',
 'addtime': '2023_05_09'}

# pubmed_raw (reserved_raw_pick)

In [50]:
collections_pubmed_raw = db.pubmed_raw

In [51]:
collections_pubmed_raw.insert_many(reserved_raw_pick)

In [52]:
reserved_raw_pick[20]

{'PMID': '26230305',
 'OWN': 'NLM',
 'STAT': 'MEDLINE',
 'DCOM': '20160121',
 'LR': '20150803',
 'IS': '2325-7237 (Electronic) 2325-7237 (Linking)',
 'VI': '5',
 'IP': '3',
 'DP': '2015 Aug 1',
 'TI': 'A Delayed Finding of a Tracheal Cartilaginous Sleeve in a Patient with Pfeiffer Syndrome Type 2 and a Complex Airway History.',
 'PG': '36-9',
 'LID': '10.1213/XAA.0000000000000175 [doi]',
 'AB': 'Persistent airway obstruction symptoms in a 2(1/2)-year-old boy with Pfeiffer syndrome were attributed to facial abnormalities, central and obstructive sleep apnea, and tracheomalacia from a vascular ring. These findings delayed the diagnosis of a tracheal cartilaginous sleeve. Life expectancy in tracheal cartilaginous sleeve is improved by tracheostomy. Tracheal cartilaginous sleeve should be considered and investigated through airway endoscopy in children with fibroblast growth factor receptor-related craniosynostosis syndromes.',
 'FAU': ['Colomb, Camille',
  'Hippard, Helena Karlberg',
  'C

# pubtator_raw_pmid

In [53]:
collection_pubtator_raw_pmid = db.pubtator_raw_pmid

In [54]:
previous_pubtator_pmid=list(collection_pubtator_raw_pmid.find({}))

In [55]:
len(previous_pubtator_pmid)

279606

In [56]:
previous_pubtator_pmids_dict={}
for i in previous_pubtator_pmid:
    previous_pubtator_pmids_dict[i.get('PMID')]=i.get('PMID')

In [57]:
current_pubtator_pmid=[]
for i in reserved_raw_pick:
    if i.get('AB'):
        if not previous_pubtator_pmids_dict.get(i.get('PMID')):
            i['addtime']=now_time
            current_pubtator_pmid.append(i.get('PMID'))

In [58]:
len(current_pubtator_pmid)

37253

In [59]:
pmids_list=[]
for i in current_pubtator_pmid:
    pmids_list.append(i)

In [60]:
len(pmids_list)

37253

In [61]:
def SubmitPMIDList(pmids,Format,now_time):
	json = {"pmids": pmids}
	r = requests.post("https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/"+Format , json = json)
	if r.status_code != 200 :
		print ("[Error]: HTTP code "+ str(r.status_code))
	else:
		with open(now_time+'_pubtator_raw.txt','a',encoding="UTF-8") as fileread:
			fileread.write(str(r.text.encode("utf-8"),encoding="UTF-8"))

In [62]:
import time
pubtator_raw=[]
for i in range(0,len(pmids_list),1000):
    SubmitPMIDList(pmids_list[i:i+1000],'biocjson',now_time)
    time.sleep(0.1)

In [63]:
ttt=open(now_time+'_pubtator_raw.txt',"r")

In [64]:
overjson=[]
for l in ttt.readlines():
    overjson.append(l.rstrip())

In [65]:
len(overjson)

37253

In [66]:
pubtator_list=copy.deepcopy(overjson)

In [67]:
pubtator_json=[]
for i in pubtator_list:
    pubtator_json.append(json.loads(i))

In [68]:
pubtator_pmid_list=[]
for i in pubtator_json:
    pubtator_pmid_list.append(i.get('id'))

In [69]:
len(pubtator_pmid_list)

37253

In [70]:
pubtator_pmid_list[0]

'37669871'

In [71]:
current_pubtator_pmid_list=[]
for i in pubtator_pmid_list:
    current_pubtator_pmid_list.append({'PMID':i,'addtime':now_time})

In [72]:
len(current_pubtator_pmid_list)

37253

In [73]:
collection_pubtator_raw_pmid.insert_many(current_pubtator_pmid_list)

# pubtator_info  (raw_pick_pubtator)

In [15]:
collections_pubtator_info = db.pubtator_info

In [75]:
filtered_pubtator_json=[]
for i in pubtator_json:
    if i.get('passages'):
        for j in i.get('passages')[1].get('annotations'):
            if j.get('infons'):
                if j.get('infons').get('identifier')=='9606':
                    filtered_pubtator_json.append(i)
                    break

In [76]:
len(filtered_pubtator_json)

29634

In [77]:
second_filtered_pubtator_json=[]
for i in filtered_pubtator_json:
    if i.get('passages'):
        for j in i.get('passages')[1].get('annotations'):
            if j.get('infons'):
                    if j.get('infons').get('type') in ('Gene', 'Chemical', 'Mutation','Disease') :
                        second_filtered_pubtator_json.append(i)
                        break

In [78]:
len(second_filtered_pubtator_json)

27003

In [79]:
entities_dict={}
for i in second_filtered_pubtator_json:
    if i.get('id'):
        entities_list=[]
        for j in i.get('passages')[1].get('annotations'):
            entities_list.append(j.get('text'))
        entities_dict[i.get('id')]=entities_list

In [80]:
second_filtered_pubtator_dict={}
for i in second_filtered_pubtator_json:
    if i.get('id'):
        second_filtered_pubtator_dict[i.get('id')]=i

In [81]:
second_filtered_pubtator_json[0]

{'_id': '37567086|None',
 'id': '37567086',
 'infons': {},
 'passages': [{'infons': {'journal': 'Int J Pediatr Otorhinolaryngol;2023Aug08; 172 111695. doi:10.1016/j.ijporl.2023.111695',
    'year': '2023',
    'type': 'title',
    'authors': 'Sud P, Munjal SK, Panda N, ',
    'section': 'Title'},
   'offset': 0,
   'text': 'Challenges faced by Indian parents in raising a child with a cochlear implant - Impact on communication outcomes.',
   'sentences': [],
   'annotations': [{'id': '1',
     'infons': {'identifier': '9606', 'type': 'Species'},
     'text': 'child',
     'locations': [{'offset': 48, 'length': 5}]}],
   'relations': []},
  {'infons': {'type': 'abstract', 'section': 'Abstract'},
   'offset': 114,
   'text': "OBJECTIVES: The objectives of the present study were to understand the parental views regarding stress, and its affect language, and auditory outcomes. The study also aims to understand the relationship between parental stress, and child's age. DESIGN, SETTING AND PA

In [82]:
from nltk.tokenize import sent_tokenize
import datetime
import re
from multiprocessing import Process, Manager, Pool
import os, time, random
def work(i,rawlength,lock):
    info=i.get('passages')[1]
    abstract=info.get('text')
    sentences=sent_tokenize(abstract)
    positions=[]
    size=len(sentences)
    start=0
    for k,j in enumerate(sentences):
        newj=j.translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                                      "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
        spanlist=re.search(newj,abstract,flags=re.M|re.I).span()
        abstract=abstract[spanlist[1]:]
        positions.append((start+spanlist[0],start+spanlist[1]))
        start+=spanlist[1]
    middict={'PMID':i.get('id'),'AB':info.get('text'),'SENTENCE':sentences,'positions':positions,'annotations':info.get('annotations'),'offset':info.get('offset')}
    lock.acquire()
    rawlength.append(middict)
    lock.release()
if __name__=='__main__':
    start=datetime.datetime.now()
    print('Parent process %s.' % os.getpid())
    p=Pool(40)
    manager=Manager()
    rawlength=manager.list()
    lock=manager.Lock()
    for i in second_filtered_pubtator_json:
        if i.get('passages'):
            p.apply_async(work, args=(i,rawlength,lock))
    print('Waiting for all subprocesses done...')
    p.close()
    p.join()
    print('All subprocesses done.')
    end=datetime.datetime.now()
    print("The running time is "+str((end-start).seconds)+"s")  

Parent process 2240.
Waiting for all subprocesses done...
All subprocesses done.
The running time is 110s


In [83]:
pubtators_list=list(rawlength)

In [84]:
len(pubtators_list)

27003

In [85]:
pubtator_raw=[]
error=[]
error2=[]
for i in pubtators_list:
    for j in i.get('annotations'):
        for k,l in enumerate(i.get('positions')):
            if j.get('locations')[0].get('offset')>=l[0]+i.get('offset') and j.get('locations')[0].get('offset')<l[1]+i.get('offset'):
                pubtator_raw.append({'id':i.get('PMID'),'AB':i.get('AB'), 'type': j.get('infons').get('type'), \
                                     'identifier':j.get('infons').get('identifier'),'text':j.get('text'),'sentence':i.get('SENTENCE')[k],\
                                     'sentenceSize':len(i.get('SENTENCE')),'sentenceIndex':k})
                break
        else:
            error.append(i)
            error2.append(j)

In [86]:
pubtator_raw_dict2AB={}
for i in pubtator_raw:
    pubtator_raw_dict2AB[i.get('id')]=i.get('AB')

In [87]:
len(error)

0

In [88]:
len(error2)

0

In [89]:
len(pubtator_raw)

421342

In [90]:
raw_pick_pubtator=[]
for i in reserved_raw_pick:
    if i.get('AB'):
        if pubtator_raw_dict2AB.get(i.get('PMID')):
            i.update({'AB':pubtator_raw_dict2AB.get(i.get('PMID'))})
            raw_pick_pubtator.append(i)

In [91]:
len(pubtator_raw_dict2AB)

27003

In [92]:
len(reserved_raw_pick)

37253

In [93]:
len(raw_pick_pubtator)

27003

In [94]:
for i in raw_pick_pubtator:
    i['addtime']=now_time

In [95]:
collections_pubtator_info.insert_many(raw_pick_pubtator)

In [96]:
len(list(collections_pubtator_info.find()))

206395

In [16]:
len(list(collections_pubtator_info.find()))

175189

# pubtator_for_compare_latest (pubtator_raw)

In [97]:
collections_pubtator_for_compare_latest=db.pubtator_for_compare_latest

In [98]:
for i in pubtator_raw:
    i['addtime']=now_time

In [99]:
collections_pubtator_for_compare_latest.insert_many(pubtator_raw)

In [100]:
len(pubtator_raw)

421342

In [101]:
pwd

'/root/hald_autoupdate/2023_10_17'

In [102]:
os.makedirs('results/step1/')
os.makedirs('results/step2/')
os.makedirs('results/step3/')
os.makedirs('results/step4/')
os.makedirs('results/step5/')

In [103]:
pickle.dump(raw_pick,open('results/step1/raw_pick.pkl','wb'))
pickle.dump(raw_pick_pubtator,open('results/step1/raw_pick_pubtator.pkl','wb'))
pickle.dump(pubtator_raw,open('results/step1/pubtator_raw.pkl','wb'))

In [104]:
pubtator_raw[0]

{'id': '37459417',
 'AB': 'More than 16 000 Americans experience spinal cord injury (SCI), resulting in chronic disability and other secondary sequelae, each year. Improvements in acute medical management have increased life expectancy. Cardiovascular disease is a leading cause of death in this population, and seems to occur earlier in individuals with SCI compared with the general population. People with SCI experience a high burden of traditional cardiovascular disease risk factors, including dyslipidemia and diabetes, and demonstrate anatomic, metabolic, and physiologic changes alongside stark reductions in physical activity after injury. They also experience multiple, complex barriers to care relating to disability and, in many cases, compounding effects of intersecting racial and socioeconomic health inequities. Given this combination of risk factors, some investigators have proposed that people with SCI are at increased risk for cardiovascular disease, beyond that associated with

In [105]:
now_time

'2023_09_30'

# Literature_Info.json

In [106]:
pd.set_option('display.max_columns', 50)

In [107]:
jcr=pd.read_csv("../JCR/jcr2022.tsv",sep="\t")

In [108]:
len(jcr)

21430

In [109]:
jcr[-10:-1]

,rank,journalTitle,abbrJournal,year,edition,issn,totalCites,journalImpactFactor,impactFactorWithoutJournalSelfCites,fiveYearImpactFactor,immediacyIndex,citableItems,citedHalfLife,citingHalfLife,eigenFactorScore,articleInfluenceScore,citesCurrent,articlesCurrent,selfCites,impactFactorSelfCites,originalResearch,jifPercentile,normEigenFactor
21420,12425,AGENDA,AGENDA,2022,NaN,0002-0796,159,0.0,0.0,0.0,0.0,0,17.8,0.0,0.00005,0.114,NaN,NaN,NaN,NaN,NaN,0.0,0.01206
21421,12425,Abakos,ABAKOS,2022,NaN,2316-9451,7,0.0,0.0,0.0,0.0,18,0.0,10.7,0.00003,0.068,NaN,NaN,NaN,NaN,NaN,0.0,0.00733
21422,12425,Adalya,ADALYA,2022,NaN,1301-2746,75,0.0,0.0,0.0,0.0,19,0.0,16.2,0.00027,0.306,NaN,NaN,NaN,NaN,NaN,0.0,0.05846
21423,12425,Anduli,ANDULI,2022,NaN,1696-0270,35,0.0,0.0,0.0,0.0,15,0.0,14.6,0.00007,0.150,NaN,NaN,NaN,NaN,NaN,0.0,0.01555
21424,12425,Anemia,ANEMIA,2022,NaN,2090-1267,502,0.0,0.0,0.0,0.0,14,7.3,7.7,0.00030,0.550,NaN,NaN,NaN,NaN,NaN,0.0,0.06589
21425,12425,Appeal,APPEAL,2022,NaN,1205-612X,56,0.0,0.0,0.0,0.0,6,0.0,4.8,0.00019,0.550,NaN,NaN,NaN,NaN,NaN,0.0,0.04134
21426,12425,Atenas,ATENAS,2022,NaN,1682-2749,60,0.0,0.0,0.0,0.0,0,0.0,0.0,0.00007,0.034,NaN,NaN,NaN,NaN,NaN,0.0,0.01561
21427,12425,Atenea,ATENEA-CONCEPC,2022,NaN,0718-0462,105,0.0,0.0,0.0,0.0,35,8.8,15.8,0.00034,0.281,NaN,NaN,NaN,NaN,NaN,0.0,0.07463
21428,12425,AusArt,AUSART,2022,NaN,2340-9134,13,0.0,0.0,0.0,0.0,32,0.0,11.8,0.00002,0.017,NaN,NaN,NaN,NaN,NaN,0.0,0.00594


In [110]:
jt2if_all={}
for i in range(len(jcr)):
    if jt2if_all.get(jcr.iloc[i,1].lower()):
        if jt2if_all.get(jcr.iloc[i,1].lower())==0.0:
            jt2if_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
    else:
        jt2if_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
jt2if_all['aging']=jt2if_all.get('aging-us')

In [111]:
jt2if_brief={}
for i in range(len(jcr)):
    if jt2if_brief.get(jcr.iloc[i,2].lower()):
        if jt2if_brief.get(jcr.iloc[i,2].lower())==0.0:
            jt2if_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
    else:
        jt2if_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
jt2if_brief['aging']=jt2if_brief.get('aging-us')

In [112]:
list(jt2if_brief.items())[:2]

[('ca-cancer j clin', 286.13), ('lancet', 202.731)]

In [113]:
len(jt2if_brief)

21429

In [114]:
jt2if5_all={}
for i in range(len(jcr)):
    if jt2if5_all.get(jcr.iloc[i,1].lower()):
        if jt2if5_all.get(jcr.iloc[i,1].lower())==0.0:
            jt2if5_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
    else:
        jt2if5_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,9]
jt2if5_all['aging']=jt2if5_all.get('aging-us')

In [115]:
jt2if5_brief={}
for i in range(len(jcr)):
    if jt2if5_brief.get(jcr.iloc[i,2].lower()):
        if jt2if5_brief.get(jcr.iloc[i,2].lower())==0.0:
            jt2if5_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
    else:
        jt2if5_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,9]
jt2if5_brief['aging']=jt2if5_brief.get('aging-us')

In [116]:
Literature_Info_list=list(collections_pubmed_raw.find())

In [117]:
len(Literature_Info_list)

339145

In [118]:
Literature_Info_dict={}
for i in Literature_Info_list:
    if jt2if_brief.get(i.get('TA').lower()):
        i['IF']=jt2if_brief.get(i.get('TA').lower())
    elif jt2if_all.get(i.get('JT').lower()):
        i['IF']=jt2if_all.get(i.get('JT').lower())
    else:
        i['IF']=0.0
    if jt2if5_brief.get(i.get('TA').lower()):
        i['IF5']=jt2if5_brief.get(i.get('TA').lower())
    elif jt2if5_all.get(i.get('JT').lower()):
        i['IF5']=jt2if5_all.get(i.get('JT').lower())
    else:
        i['IF5']=0.0
    Literature_Info_dict[i.get('PMID')]={
    'PMID':i.get('PMID'),
        'TI':i.get('TI'),
        'AB':i.get('AB'),
        'IF':i.get('IF'),
        'IF5':i.get('IF5'),
        'AU':i.get('AU'),
        'FAU':i.get('FAU'),
        'AD':i.get('AD'),
        'PT':i.get('PT'),
        'DP':i.get('DP'),
        'PL':i.get('PL'),
        'JT':i.get('JT'),
        'TA':i.get('TA'),
        'SO':i.get('SO')
    }

In [119]:
list(Literature_Info_dict.items())[10]

('35795148',
 {'PMID': '35795148',
  'TI': 'Sirtuin 1 in Chronic Kidney Disease and Therapeutic Potential of Targeting Sirtuin 1.',
  'AB': 'The incidence and prevalence of chronic kidney disease (CKD) continue to increase worldwide remaining as a major public health burden. CKD eventually progresses to end-stage kidney failure and patients with CKD have high morbidity and mortality. Sirtuin 1 (SIRT1), a NAD+-dependent deacetylases, has significant renal protective effects through its regulation of fibrosis, apoptosis, and senescence, oxidative stress, inflammation and aging process. The renal protective effects of Sirt1 have been described in many kidney diseases such as diabetic kidney disease and HIV-related kidney disease. SIRT1 also has protective effects against vascular calcification and therefore could be developed as a therapy for both CKD and CKD complications. In this narrative review, we will give an overview of the recent progress on the role of SIRT1 and its downstream pa

In [120]:
json.dump(Literature_Info_dict,open('results/step5/Literature_Info.json','w'))

In [121]:
len(Literature_Info_dict)

339145